<a href="https://colab.research.google.com/github/adarshtomar333/Hand-Written-Character-Recognition/blob/main/hcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']  = '/content'
!kaggle datasets download -d vaibhao/handwritten-characters
!unzip \*.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/Validation/F/167.jpg  
  inflating: dataset/Validation/F/168.jpg  
  inflating: dataset/Validation/F/169.jpg  
  inflating: dataset/Validation/F/17.jpg  
  inflating: dataset/Validation/F/170.jpg  
  inflating: dataset/Validation/F/171.jpg  
  inflating: dataset/Validation/F/172.jpg  
  inflating: dataset/Validation/F/173.jpg  
  inflating: dataset/Validation/F/174.jpg  
  inflating: dataset/Validation/F/175.jpg  
  inflating: dataset/Validation/F/176.jpg  
  inflating: dataset/Validation/F/177.jpg  
  inflating: dataset/Validation/F/178.jpg  
  inflating: dataset/Validation/F/179.jpg  
  inflating: dataset/Validation/F/18.jpg  
  inflating: dataset/Validation/F/180.jpg  
  inflating: dataset/Validation/F/181.jpg  
  inflating: dataset/Validation/F/182.jpg  
  inflating: dataset/Validation/F/183.jpg  
  inflating: dataset/Validation/F/184.jpg  
  inflating: dataset/Validation/F/185.jpg  
  inflating: dataset/Valida

In [ ]:
# Verify if the directories exist and print some sample files
train_image_dir = '/content/dataset/Train'
test_image_dir = '/content/dataset/Validation'

if not os.path.exists(train_image_dir) or not os.path.exists(test_image_dir):
    raise Exception("Image directories not found. Please check the paths.")

print("Sample training images:", os.listdir(train_image_dir)[:5])
print("Sample testing images:", os.listdir(test_image_dir)[:5])

Sample training images: ['3', '@', 'B', '8', 'P']
Sample testing images: ['3', '@', 'B', '8', 'P']


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Define the paths to the directories
train_dir = '/content/dataset/Train'
test_dir = '/content/dataset/Validation'

# Load and preprocess the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True
)

# Load and preprocess the testing dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True
)

# Extract class names before mapping
class_names = train_dataset.class_names
num_classes = len(class_names)

# Normalize the images to the range [0, 1]
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_dataset = train_dataset.map(normalize_img)
test_dataset = test_dataset.map(normalize_img)

# Convert datasets to use in the model
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Found 834036 files belonging to 39 classes.
Found 22524 files belonging to 39 classes.


In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
26064/26064 [==============================] - 939s 36ms/step - loss: 0.2455 - accuracy: 0.9206 - val_loss: 0.2001 - val_accuracy: 0.9374
Epoch 2/10
26064/26064 [==============================] - 927s 36ms/step - loss: 0.1669 - accuracy: 0.9435 - val_loss: 0.1836 - val_accuracy: 0.9416
Epoch 3/10
26064/26064 [==============================] - 924s 35ms/step - loss: 0.1538 - accuracy: 0.9475 - val_loss: 0.1924 - val_accuracy: 0.9401
Epoch 4/10
26064/26064 [==============================] - 935s 36ms/step - loss: 0.1467 - accuracy: 0.9492 - val_loss: 0.1819 - val_accuracy: 0.9437
Epoch 5/10
26064/26064 [==============================] - 913s 35ms/step - loss: 0.1424 - accuracy: 0.9507 - val_loss: 0.1858 - val_accuracy: 0.9444
Epoch 6/10
26064/26064 [==============================] - 924s 35ms/step - loss: 0.1392 - accuracy: 0.9515 - val_loss: 0.1831 - val_accuracy: 0.9461
Epoch 7/10
26064/26064 [==============================] - 916s 35ms/step - loss: 0.1369 - accuracy: 0.9521

In [2]:
# Visualize training results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Save the model to an .h5 file
model.save('handwritten_character_recognition_model.h5')

# Save the model architecture to a JSON file
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an HDF5 file
model.save_weights("model_weights.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Import the files module for downloading files
from google.colab import files

# Download the .h5 file
files.download('handwritten_character_recognition_model.h5')

# Download the JSON file
files.download('model_architecture.json')

# Download the weights file
files.download('model_weights.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>